In [1]:
! pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=0381dfadf8a00bd2c9f9eb17d94b921da30a6d2c0694661c7bb405d1097feedf
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [2]:
import yfinance as yf
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import ParameterGrid
from backtesting import Backtest, Strategy
from backtesting.test import SMA

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-cons

In [3]:
# Pobieranie danych historycznych dla MSFT
ticker = 'MSFT'
data = yf.download(ticker, start='2020-01-01', end='2024-01-01')
data.reset_index(inplace=True)
data = data[['Date', 'Close']]
data.columns = ['ds', 'y']

[*********************100%%**********************]  1 of 1 completed


In [4]:
# Parametry do optymalizacji
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

In [5]:
# Wybór najlepszych parametrów do modelu
best_params = None
best_mae = float('inf')

for params in ParameterGrid(param_grid):
    model = Prophet(**params)
    model.fit(data)

    future = model.make_future_dataframe(periods=126)  # Zakładając prognozowanie na 2024-01-01 do 2024-05-06
    forecast = model.predict(future)

    forecast_actual = forecast[['ds', 'yhat']].merge(data, on='ds', how='inner')
    mae = mean_absolute_error(forecast_actual['y'], forecast_actual['yhat'])

    if mae < best_mae:
        best_mae = mae
        best_params = params

print(f"Best parameters: {best_params}")

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_43af70/tr2huvdc.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_43af70/lvr8zzip.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=90622', 'data', 'file=/tmp/tmpj_43af70/tr2huvdc.json', 'init=/tmp/tmpj_43af70/lvr8zzip.json', 'output', 'file=/tmp/tmpj_43af70/prophet_model31iawmd4/prophet_model-20240604143630.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:36:30 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:36:31 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_

Best parameters: {'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 0.1}


In [6]:
# Stworzenie modelu z najlepszymi parametrami
model = Prophet(**best_params)
model.fit(data)
future = pd.date_range(start='2024-01-02', end='2024-05-02')
future_df = pd.DataFrame(future, columns=['ds'])
forecast = model.predict(future_df)
forecast

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_43af70/tbk3m6p3.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_43af70/v15ns531.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=37794', 'data', 'file=/tmp/tmpj_43af70/tbk3m6p3.json', 'init=/tmp/tmpj_43af70/v15ns531.json', 'output', 'file=/tmp/tmpj_43af70/prophet_modelo6mrjnwy/prophet_model-20240604143701.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:37:01 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:37:02 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2024-01-02,390.790744,370.111384,389.794573,390.790744,390.790744,-10.733004,-10.733004,-10.733004,-0.253150,-0.253150,-0.253150,-10.479854,-10.479854,-10.479854,0.0,0.0,0.0,380.057740
1,2024-01-03,391.216019,370.507879,389.814425,391.216019,391.216019,-10.951894,-10.951894,-10.951894,0.163392,0.163392,0.163392,-11.115285,-11.115285,-11.115285,0.0,0.0,0.0,380.264125
2,2024-01-04,391.641294,370.494944,389.408627,391.641294,391.641294,-11.681048,-11.681048,-11.681048,0.114195,0.114195,0.114195,-11.795243,-11.795243,-11.795243,0.0,0.0,0.0,379.960245
3,2024-01-05,392.066569,370.349747,389.095104,392.066569,392.066569,-12.316839,-12.316839,-12.316839,0.190785,0.190785,0.190785,-12.507624,-12.507624,-12.507624,0.0,0.0,0.0,379.749730
4,2024-01-06,392.491844,369.595622,388.436285,392.491844,392.491844,-13.294315,-13.294315,-13.294315,-0.055524,-0.055524,-0.055524,-13.238790,-13.238790,-13.238790,0.0,0.0,0.0,379.197529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,2024-04-28,440.547914,392.274598,522.949748,375.967678,503.020338,18.408280,18.408280,18.408280,-0.055524,-0.055524,-0.055524,18.463804,18.463804,18.463804,0.0,0.0,0.0,458.956193
118,2024-04-29,440.973189,392.111344,523.408639,375.280369,504.442772,18.341318,18.341318,18.341318,-0.104173,-0.104173,-0.104173,18.445491,18.445491,18.445491,0.0,0.0,0.0,459.314507
119,2024-04-30,441.398463,392.537332,526.851514,374.593060,505.857840,18.178685,18.178685,18.178685,-0.253150,-0.253150,-0.253150,18.431835,18.431835,18.431835,0.0,0.0,0.0,459.577148
120,2024-05-01,441.823738,393.017192,528.571406,373.905752,507.043420,18.590526,18.590526,18.590526,0.163392,0.163392,0.163392,18.427134,18.427134,18.427134,0.0,0.0,0.0,460.414264


In [7]:
# Generowanie sygnałów handlowych
def generate_signals(forecast, threshold=0.1):
    forecast['signal'] = 0
    forecast['signal'][forecast['yhat'].diff() > threshold] = 1
    forecast['signal'][forecast['yhat'].diff() < -threshold] = -1
    return forecast

signals = generate_signals(forecast)
print(signals)

            ds       trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
0   2024-01-02  390.790744  370.111384  389.794573   390.790744   390.790744   
1   2024-01-03  391.216019  370.507879  389.814425   391.216019   391.216019   
2   2024-01-04  391.641294  370.494944  389.408627   391.641294   391.641294   
3   2024-01-05  392.066569  370.349747  389.095104   392.066569   392.066569   
4   2024-01-06  392.491844  369.595622  388.436285   392.491844   392.491844   
..         ...         ...         ...         ...          ...          ...   
117 2024-04-28  440.547914  392.274598  522.949748   375.967678   503.020338   
118 2024-04-29  440.973189  392.111344  523.408639   375.280369   504.442772   
119 2024-04-30  441.398463  392.537332  526.851514   374.593060   505.857840   
120 2024-05-01  441.823738  393.017192  528.571406   373.905752   507.043420   
121 2024-05-02  442.249013  392.815797  526.014159   373.221282   508.229000   

     additive_terms  additive_terms_low

<ipython-input-7-cc11e9b9171a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast['signal'][forecast['yhat'].diff() > threshold] = 1
<ipython-input-7-cc11e9b9171a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast['signal'][forecast['yhat'].diff() < -threshold] = -1


In [8]:
# Backtesting
class ProphetStrategy(Strategy):
    def init(self):
        self.forecast = signals.set_index('ds')

    def next(self):
        date = self.data.index[-1]

        if date in self.forecast.index:
            signal = self.forecast.loc[date]['signal']

            if signal == 1:
                self.buy(sl=self.data.Close[-1] * 0.95, tp=self.data.Close[-1] * 1.06)  # Stop-loss 5%, Take-profit 6%
            elif signal == -1:
                self.sell(sl=self.data.Close[-1] * 1.05, tp=self.data.Close[-1] * 0.95)  # Stop-loss 5%, Take-profit 5%

In [9]:
# Przygotowanie danych do backtestu
bt_data = yf.download(ticker, start='2024-01-02', end='2024-05-02')

bt = Backtest(bt_data, ProphetStrategy, cash=10000)
stats = bt.run()
bt.plot()

print(stats)

[*********************100%%**********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2024-01-02 00:00:00
End                       2024-05-01 00:00:00
Duration                    120 days 00:00:00
Exposure Time [%]                   97.619048
Equity Final [$]                 10207.048867
Equity Peak [$]                  11896.876634
Return [%]                           2.070489
Buy & Hold Return [%]                6.490147
Return (Ann.) [%]                    6.340961
Volatility (Ann.) [%]               23.376011
Sharpe Ratio                         0.271259
Sortino Ratio                        0.383783
Calmar Ratio                          0.41221
Max. Drawdown [%]                  -15.382846
Avg. Drawdown [%]                   -3.232724
Max. Drawdown Duration       41 days 00:00:00
Avg. Drawdown Duration       13 days 00:00:00
# Trades                                    5
Win Rate [%]                             40.0
Best Trade [%]                       6.625645
Worst Trade [%]                     -4.745621
Avg. Trade [%]                    